In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Read Churn Dataset and get summary of basic statistics
df=pd.read_csv('./Resources/bin_10_clean.csv')
df.describe()

,Unnamed: 0,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,bin_age,gender_0,gender_1
count,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000,9589.000000
mean,4794.000000,650.287830,37.660444,5.013974,76552.236761,1.530921,0.705809,0.502138,100200.278401,0.202628,2.210241,0.453958,0.546042
std,2768.250199,96.660575,8.644055,2.888560,62422.133272,0.581498,0.455702,0.500022,57500.956105,0.401979,0.901729,0.497902,0.497902
min,0.000000,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,2397.000000,583.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,51226.320000,0.000000,2.000000,0.000000,0.000000
50%,4794.000000,652.000000,37.000000,5.000000,97266.100000,1.000000,1.000000,1.000000,100335.550000,0.000000,2.000000,0.000000,1.000000
75%,7191.000000,717.000000,43.000000,7.000000,127661.690000,2.000000,1.000000,1.000000,149458.730000,0.000000,3.000000,1.000000,1.000000
max,9588.000000,850.000000,61.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000,5.000000,1.000000,1.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       9589 non-null   int64  
 1   CreditScore      9589 non-null   int64  
 2   Geography        9589 non-null   object 
 3   Age              9589 non-null   int64  
 4   Tenure           9589 non-null   int64  
 5   Balance          9589 non-null   float64
 6   NumOfProducts    9589 non-null   int64  
 7   HasCrCard        9589 non-null   int64  
 8   IsActiveMember   9589 non-null   int64  
 9   EstimatedSalary  9589 non-null   float64
 10  Exited           9589 non-null   int64  
 11  bin_age          9589 non-null   int64  
 12  gender_0         9589 non-null   int64  
 13  gender_1         9589 non-null   int64  
dtypes: float64(2), int64(11), object(1)
memory usage: 1.0+ MB


### Smoteenn Remove Age Outliers

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

In [5]:
from pathlib import Path
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [12]:
#Create Features
X = pd.get_dummies(df, columns=['Geography'])
X = X.drop(["Exited"], axis=1)

# Create our target
y = df["Exited"]

In [13]:
# Check the balance of our target values
y.value_counts()

0    7646
1    1943
Name: Exited, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
Counter(y_train)

Counter({0: 5740, 1: 1451})

In [15]:
# Standardize the data with StandardScaler().
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Combined over and under sampling for imbalanced classification
from imblearn.combine import SMOTEENN

# define sampling
comb_sample = SMOTEENN(random_state=42)
X_resample, y_resample = comb_sample.fit_resample(X_train, y_train)

# Check if SMOTE-ENN were properly applied
Counter(y_resample)

Counter({0: 3804, 1: 5080})

### Fit Different Module Define

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

### AdaBoost Model with SMOTEENN

In [18]:
# Initiate the model
smoteenn_adb = AdaBoostClassifier()

# Fit the model
smoteenn_adb_model = smoteenn_adb.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_adb_model.predict(X_test)

In [19]:
# Calculated the balanced accuracy score
smoteenn_adb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_adb_accuracy

0.7574081420247571

In [20]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1419,487
Actual Exited,113,379


In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn adb classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn adb classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.74      0.77      0.83      0.76      0.57      1906
          1       0.44      0.77      0.74      0.56      0.76      0.57       492

avg / total       0.83      0.75      0.77      0.77      0.76      0.57      2398



### KNN with SMOTEENN

In [22]:
# Initiate the model
smoteenn_knn = KNeighborsClassifier()

# Fit the model
smoteenn_knn_model = smoteenn_knn.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_knn_model.predict(X_test)

In [23]:
# Calculated the balanced accuracy score
smoteenn_knn_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_knn_accuracy

0.7094391694179272

In [24]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1302,604
Actual Exited,130,362


In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn knn classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn knn classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.68      0.74      0.78      0.71      0.50      1906
          1       0.37      0.74      0.68      0.50      0.71      0.51       492

avg / total       0.80      0.69      0.72      0.72      0.71      0.50      2398



### SVM with SMOTEENN

In [26]:
# Initiate the model
smoteenn_svc = SVC(kernel='rbf',probability=True)

# Fit the model
smoteenn_svc_model = smoteenn_svc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_svc_model.predict(X_test)

In [27]:
# Calculated the balanced accuracy score
smoteenn_svc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_svc_accuracy

0.7481914194797772

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1349,557
Actual Exited,104,388


In [29]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn svc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn svc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.71      0.79      0.80      0.75      0.55      1906
          1       0.41      0.79      0.71      0.54      0.75      0.56       492

avg / total       0.82      0.72      0.77      0.75      0.75      0.56      2398



### Decision Tree with SMOTEENN

In [30]:
# Initiate the model
smoteenn_tree = DecisionTreeClassifier()

# Fit the model
smoteenn_tree_model = smoteenn_tree.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_tree_model.predict(X_test)

In [31]:
# Calculated the balanced accuracy score
smoteenn_tree_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_tree_accuracy

0.7504308175295814

In [32]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1497,409
Actual Exited,140,352


In [33]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn tree classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn tree classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.79      0.72      0.85      0.75      0.57      1906
          1       0.46      0.72      0.79      0.56      0.75      0.56       492

avg / total       0.82      0.77      0.73      0.79      0.75      0.56      2398



### Random Forest with SMOTEENN

In [34]:
# Initiate the model
smoteenn_rfc = RandomForestClassifier()

# Fit the model
smoteenn_rfc_model = smoteenn_rfc.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred = smoteenn_rfc_model.predict(X_test)

In [35]:
# Calculated the balanced accuracy score
smoteenn_rfc_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_rfc_accuracy

0.7781673619464422

In [36]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1533,373
Actual Exited,122,370


In [37]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn rfc classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn rfc classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.80      0.75      0.86      0.78      0.61      1906
          1       0.50      0.75      0.80      0.60      0.78      0.60       492

avg / total       0.84      0.79      0.76      0.81      0.78      0.61      2398



### XGBoost Classifier with SMOTEENN

In [38]:
# Initiate the model
smoteenn_xgb = XGBClassifier()
# Fit the model
smoteenn_xgb_model =smoteenn_xgb.fit(X_resample, y_resample.ravel())
# Make Predictions
y_pred = smoteenn_xgb_model.predict(X_test)

In [39]:
# Calculated the balanced accuracy score
smoteenn_xgb_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_xgb_accuracy

0.7757797370733414

In [40]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1613,293
Actual Exited,145,347


In [41]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn xgb classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn xgb classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.85      0.71      0.88      0.77      0.61      1906
          1       0.54      0.71      0.85      0.61      0.77      0.59       492

avg / total       0.84      0.82      0.73      0.83      0.77      0.60      2398



### Logistic Regression with SMOTEENN

In [42]:
# Initiate the model
smoteenn_lm = LogisticRegression()

# Fit the model
smoteenn_lm_model = smoteenn_lm.fit(X_resample, y_resample.ravel())

# Make Predictions
y_pred=smoteenn_lm_model.predict(X_test)

In [43]:
# Calculated the balanced accuracy score
smoteenn_lm_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_lm_accuracy

0.7074311758332693

In [44]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual Active", "Actual Exited"], columns=["Predicted Active", "Predicted Exited"])

cm_df

,Predicted Active,Predicted Exited
Actual Active,1182,724
Actual Exited,101,391


In [45]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(f'smoteenn lm classification report')
print(classification_report_imbalanced(y_test, y_pred))

smoteenn lm classification report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.62      0.79      0.74      0.70      0.48      1906
          1       0.35      0.79      0.62      0.49      0.70      0.50       492

avg / total       0.80      0.66      0.76      0.69      0.70      0.49      2398

